## Deep Determenistic Policy Gradients
Work in progress nut it will be finished soon enough

Special thanks to KnightofK9

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pickle

cuda = torch.device('cuda')
frame_size = 10

In [3]:
ratings = pd.read_csv('../data/ml-20m/ratings.csv')
movies = pickle.load(open('../data/infos.pytorch', 'rb'))

In [4]:
# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]
ratings = ratings[ratings["userId"].isin(users.index)]
ratings = ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")

In [5]:
for i in movies.keys():
    movies[i] = movies[i].to(cuda)

# Model

In [ ]:
embeddings = nn.Sequential(nn.Linear(2591, 64), nn.Tanh()).to(cuda)

In [50]:
class StateRepresentation(nn.Module):
    def __init__(self):
        super(StateRepresentation, self).__init__()
        self.lin = nn.Sequential(
            # 64 - embed size, 1 - rating size
            nn.Linear(frame_size * (64 + 1), 128),
            nn.Tanh(),
        )
        
    def forward(self, info, ratings):
        # raw_size - size of the raw movie info. Constant = 2591
        # embed_size - size of an ebedded movie. Constant = 64
        # raw -> embed via embeddings module defined above
        # input: currently info is batch_size x frame_size x raw_size
        # step 1: transform info to (batch_size * frame_size) x raw_size
        info = info.view(-1, 2591)
        # step 2: apply embeddings, map info to (batch_size * frame_size) x embed_size
        info = embeddings(info)
        # step 3: transform info back to batch_size x frame_size x raw_size
        info = info.view(batch_size, frame_size, 64)
        # step 4: tramsform info to batch_size x (frame_size * embed_size)
        info = info.view(batch_size, frame_size * 64)
        # step 5: stack info with ratings. stacked: batch_size x (embed_size + 1)
        stacked = torch.cat([info, ratings], 1)
        # step 6: apply state represemtation module
        state = self.lin(stacked)
        return state

In [51]:
sr = StateRepresentation().to(cuda)

# Training

In [ ]:

def prepare_batch(batch):
    watched_infos = []
    watched_rating = []
    chosen_movie = []
    chosen_rating = []
    
    for b in batch:
        watched_infos.append(b[0][0])
        watched_rating.append(torch.from_numpy(b[0][1]))
        chosen_movie.append(b[1])
        chosen_rating.append(b[2])
                    
    watched_infos = torch.stack(watched_infos).to(cuda)
    watched_rating = torch.stack(watched_rating).float().to(cuda)
    chosen_movie = torch.stack(chosen_movie).to(cuda)
    chosen_rating = torch.tensor(chosen_rating).to(cuda)
    
    return (watched_infos, watched_rating), chosen_movie, chosen_rating

batch_bar = tqdm(total=len(users))
batch = []
batch_size = 100
for user, df in ratings.groupby(level=0):
    batch_bar.update(1)
    size = max(len(df) - frame_size, 0)
    for idx in range(0, size):
        
        if np.random.rand() < 0.8:  # intake percents
            continue
            
        user_ratings = df[idx:frame_size + idx + 1]
        user_ratings = user_ratings[["movieId", "rating"]].values

        chosen_movie = user_ratings[:, 0][-1] 
        chosen_movie = movies[chosen_movie] # action
        chosen_rating = user_ratings[:, 1][-1] # reward
        films_watched = user_ratings[:, 0][:-1] 
        watched_rating = user_ratings[:, 1][:-1] # state
        watched_infos = [movies[i] for i in films_watched] # state
        watched_infos = torch.stack(watched_infos)    
        # state action reward
        batch.append([(watched_infos, watched_rating), chosen_movie, chosen_rating])
                
        if len(batch) >= batch_size:
            # train here
            batch = prepare_batch(batch)
            sr(*batch[0]).size()
            batch = []